<a href="https://colab.research.google.com/github/morenoyuri/pln/blob/main/Projeto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 3

Nesse projeto temos como objetivo desenvolver um programa para recomendar livros baseados na similaridade de textos na Wikipedia.

In [ ]:
import pandas as pd

In [ ]:
!pip install wikipedia

O código abaixo pega os artigos da Wikipedia, insere no dicionário d e depois adicionará no DataFrame df.

In [99]:
# Pegando artigos da wikipedia
import wikipedia

books = []

books.append(wikipedia.page("Harry Potter and the Philosopher's Stone", auto_suggest=False))
books.append(wikipedia.page("Harry Potter and the Chamber of Secrets", auto_suggest=False))
books.append(wikipedia.page("Harry Potter and the Prisoner of Azkaban", auto_suggest=False))
books.append(wikipedia.page("Harry Potter and the Goblet of Fire", auto_suggest=False))
books.append(wikipedia.page("Harry Potter and the Order of the Phoenix", auto_suggest=False))
books.append(wikipedia.page("Harry Potter and the Half-Blood Prince", auto_suggest=False))
books.append(wikipedia.page("Harry Potter and the Deathly Hallows", auto_suggest=False))

books.append(wikipedia.page("The Hobbit", auto_suggest=False))
books.append(wikipedia.page("The Fellowship of the Ring", auto_suggest=False))
books.append(wikipedia.page("The Two Towers", auto_suggest=False))
books.append(wikipedia.page("The Return of the King", auto_suggest=False))

books.append(wikipedia.page("A Game of Thrones", auto_suggest=False))
books.append(wikipedia.page("A Clash of Kings", auto_suggest=False))
books.append(wikipedia.page("A Storm of Swords", auto_suggest=False))
books.append(wikipedia.page("A Feast for Crows", auto_suggest=False))
books.append(wikipedia.page("A Dance with Dragons", auto_suggest=False))
books.append(wikipedia.page("The Winds of Winter", auto_suggest=False))

books.append(wikipedia.page("The Lion, the Witch and the Wardrobe", auto_suggest=False))
books.append(wikipedia.page("Prince Caspian", auto_suggest=False))
books.append(wikipedia.page("The Voyage of the Dawn Treader", auto_suggest=False))
books.append(wikipedia.page("The Silver Chair", auto_suggest=False))
books.append(wikipedia.page("The Horse and His Boy", auto_suggest=False))
books.append(wikipedia.page("The Magician's Nephew", auto_suggest=False))
books.append(wikipedia.page("The Last Battle", auto_suggest=False))

books.append(wikipedia.page("Eragon", auto_suggest=False))
books.append(wikipedia.page("Eldest", auto_suggest=False))
books.append(wikipedia.page("Brisingr", auto_suggest=False))
books.append(wikipedia.page("Inheritance (Paolini novel)", auto_suggest=False))

books.append(wikipedia.page("The Hunger Games (novel)", auto_suggest=False))
books.append(wikipedia.page("Catching Fire", auto_suggest=False))
books.append(wikipedia.page("Mockingjay", auto_suggest=False))
books.append(wikipedia.page("The Ballad of Songbirds and Snakes", auto_suggest=False))

books.append(wikipedia.page("In Search of Lost Time", auto_suggest=False))
books.append(wikipedia.page("Ulysses (novel)", auto_suggest=False))
books.append(wikipedia.page("Don Quixote", auto_suggest=False))
books.append(wikipedia.page("One Hundred Years of Solitude", auto_suggest=False))
books.append(wikipedia.page("The Great Gatsby", auto_suggest=False))
books.append(wikipedia.page("Moby Dick", auto_suggest=False))
books.append(wikipedia.page("War and Peace", auto_suggest=False))
books.append(wikipedia.page("Hamlet", auto_suggest=False))
books.append(wikipedia.page("Alice's Adventures in Wonderland", auto_suggest=False))
books.append(wikipedia.page("Rebecca", auto_suggest=False))

d = {"title":[], "content":[]}
for book in books:
  d["title"].append(book.original_title)
  d["content"].append(book.content)
df = pd.DataFrame(data=d)

In [100]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [102]:
import re
import numpy as np

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    # remove all single characters
    doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc)
    # remove single characters from the start
    doc = re.sub(r'\^[a-zA-Z]\s+', ' ', doc)
    # substituting multiple spaces with single space
    doc = re.sub(r'\s+', ' ', doc, flags=re.I)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['content']))
len(norm_corpus)

42

O código acima faz a normalização dos artigos pegados anteriormente, o código usado é o mesmo da aula sobre Comparação de textos.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(42, 11602)

Utilizando de uni-grama e bi-gramas, vetorizamos os documentos para poder calcular a similaridade dos textos utilizando similaridade por cosseno.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,1.000000,0.642237,0.569374,0.641148,0.647236,0.582632,0.674721,0.099039,0.059361,0.062289,0.053559,0.072001,0.045155,0.057598,0.050837,0.055305,0.049176,0.110601,0.047512,0.055894,0.044363,0.040851,0.075499,0.039405,0.074671,0.067213,0.072610,0.053266,0.087130,0.054628,0.082420,0.030741,0.087362,0.075232,0.056946,0.093784,0.074500,0.089878,0.080616,0.041441,0.049714,0.029064
1,0.642237,1.000000,0.514355,0.557032,0.581257,0.562610,0.581728,0.062421,0.033799,0.034798,0.032633,0.051360,0.042229,0.049748,0.034059,0.041666,0.033753,0.058805,0.032967,0.032631,0.029876,0.027169,0.043835,0.020913,0.055054,0.051524,0.047794,0.042132,0.056814,0.037859,0.053885,0.030165,0.056267,0.043799,0.032423,0.053850,0.068909,0.051160,0.051467,0.029872,0.030686,0.019556
2,0.569374,0.514355,1.000000,0.533760,0.534481,0.475627,0.507930,0.072952,0.056537,0.057959,0.041704,0.073190,0.063436,0.070249,0.049844,0.051940,0.043043,0.065851,0.037925,0.038475,0.032914,0.027975,0.043214,0.030499,0.064673,0.070918,0.064171,0.045185,0.082110,0.057061,0.073198,0.039680,0.055913,0.053211,0.031894,0.053588,0.049889,0.061945,0.053983,0.024861,0.035774,0.014028
3,0.641148,0.557032,0.533760,1.000000,0.621224,0.551239,0.604270,0.067875,0.054652,0.044786,0.047759,0.061678,0.050474,0.071663,0.043251,0.047654,0.046295,0.065546,0.037693,0.044041,0.039966,0.030459,0.051122,0.022776,0.071434,0.065862,0.059722,0.052323,0.068810,0.061009,0.074111,0.032865,0.066053,0.053104,0.035413,0.061169,0.059762,0.059150,0.054530,0.030035,0.032535,0.025213
4,0.647236,0.581257,0.534481,0.621224,1.000000,0.640099,0.678280,0.050203,0.034302,0.027846,0.033338,0.046159,0.035023,0.049385,0.035282,0.041642,0.032753,0.048451,0.028646,0.030853,0.025398,0.024106,0.039329,0.022723,0.061724,0.048251,0.049052,0.044074,0.058672,0.043173,0.062315,0.023193,0.047163,0.042726,0.035383,0.052237,0.041610,0.045356,0.050038,0.021275,0.027310,0.016734


Assim como visto em aula, as linhas de código acima realizam a similaridade por cosseno e verifica os termos similares, mas ao invés de termos, com documentos. Os documentos são strings com valores TF-IDF.
Abaixo vamos capturar os títulos dos filmes para usar na função.

In [ ]:
book_list = df['title'].values
book_list, book_list.shape

(array(["Harry Potter and the Philosopher's Stone",
        'Harry Potter and the Chamber of Secrets',
        'Harry Potter and the Prisoner of Azkaban',
        'Harry Potter and the Goblet of Fire',
        'Harry Potter and the Order of the Phoenix',
        'Harry Potter and the Half-Blood Prince',
        'Harry Potter and the Deathly Hallows', 'The Hobbit',
        'The Fellowship of the Ring', 'The Two Towers',
        'The Return of the King', 'A Game of Thrones', 'A Clash of Kings',
        'A Storm of Swords', 'A Feast for Crows', 'A Dance with Dragons',
        'The Winds of Winter', 'The Lion, the Witch and the Wardrobe',
        'Prince Caspian', 'The Voyage of the Dawn Treader',
        'The Silver Chair', 'The Horse and His Boy',
        "The Magician's Nephew", 'The Last Battle', 'Eragon', 'Eldest',
        'Brisingr', 'Inheritance (Paolini novel)',
        'The Hunger Games (novel)', 'Catching Fire', 'Mockingjay',
        'The Ballad of Songbirds and Snakes', 'In Sear

# Construindo de um recomendador de livros

Juntando todos os conceitos vamos fazer o recomendador de livros

In [ ]:
def book_recommender(book_title, book=book_list, doc_sims=doc_sim_df, quantity=5):
    quantity = quantity + 1
    # find book id
    book_idx = np.where(book == book_title)[0][0]
    # get book similarities
    book_similarities = doc_sims.iloc[book_idx].values
    # get top n similar book IDs
    similar_book_idxs = np.argsort(-book_similarities)[1:quantity]
    # get top n books
    similar_books = book[similar_book_idxs]
    # return the top n books
    return similar_books

Nos próximos trechos de código vamos pegar todos os livros e pegar os mais recomendados para cada um.

In [ ]:
all_books = df['title'].to_list()
print(all_books)

["Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Half-Blood Prince', 'Harry Potter and the Deathly Hallows', 'The Hobbit', 'The Fellowship of the Ring', 'The Two Towers', 'The Return of the King', 'A Game of Thrones', 'A Clash of Kings', 'A Storm of Swords', 'A Feast for Crows', 'A Dance with Dragons', 'The Winds of Winter', 'The Lion, the Witch and the Wardrobe', 'Prince Caspian', 'The Voyage of the Dawn Treader', 'The Silver Chair', 'The Horse and His Boy', "The Magician's Nephew", 'The Last Battle', 'Eragon', 'Eldest', 'Brisingr', 'Inheritance (Paolini novel)', 'The Hunger Games (novel)', 'Catching Fire', 'Mockingjay', 'The Ballad of Songbirds and Snakes', 'In Search of Lost Time', 'Ulysses (novel)', 'Don Quixote', 'One Hundred Years of Solitude', 'The Great Gatsby', 'Moby-Dick', 'War and Peace', 

In [103]:
for book in all_books:
  print("Book: ", book)
  print("Top 10 recommended Books: ", book_recommender(book_title=book, quantity=10))
  print()

Book:  Harry Potter and the Philosopher's Stone
Top 10 recommended Books:  ['Harry Potter and the Deathly Hallows'
 'Harry Potter and the Order of the Phoenix'
 'Harry Potter and the Chamber of Secrets'
 'Harry Potter and the Goblet of Fire'
 'Harry Potter and the Half-Blood Prince'
 'Harry Potter and the Prisoner of Azkaban'
 'The Lion, the Witch and the Wardrobe' 'The Hobbit'
 'One Hundred Years of Solitude' 'Moby-Dick']

Book:  Harry Potter and the Chamber of Secrets
Top 10 recommended Books:  ["Harry Potter and the Philosopher's Stone"
 'Harry Potter and the Deathly Hallows'
 'Harry Potter and the Order of the Phoenix'
 'Harry Potter and the Half-Blood Prince'
 'Harry Potter and the Goblet of Fire'
 'Harry Potter and the Prisoner of Azkaban' 'The Great Gatsby'
 'The Hobbit' 'The Lion, the Witch and the Wardrobe'
 'The Hunger Games (novel)']

Book:  Harry Potter and the Prisoner of Azkaban
Top 10 recommended Books:  ["Harry Potter and the Philosopher's Stone"
 'Harry Potter and the 